In [91]:
from methods import load_data, select_route_data, group_nodes, group_links, obtain_routes, export_results, select_volume_data, routes_volume_join, drop_duplicates
import pathlib
import json

In [2]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [3]:
#Load raw data
strategic_raw_data, qgis_table = load_data(file, file)

In [4]:
nodes = select_route_data(strategic_raw_data)
ogv_nodes = select_route_data(strategic_raw_data, ogv=True) #Extract routes for user class 9 (OGVs)

In [5]:
nodes_grouped=group_nodes(nodes)
ogv_nodes_grouped=group_nodes(ogv_nodes)

In [6]:
links = group_links(nodes_grouped)
ogv_links = group_links(ogv_nodes_grouped)

In [7]:
routes = obtain_routes(links, qgis_table)
ogv_routes=obtain_routes(ogv_links, qgis_table)

Separate gis sheet from same file

In [8]:
all_routes = routes + ogv_routes

In [9]:
volumes=select_volume_data(strategic_raw_data)

In [10]:
routes = routes_volume_join(routes, volumes)
ogv_routes = routes_volume_join(ogv_routes, volumes)

In [11]:
unique_routes = drop_duplicates(routes)
ogv_unique_routes = drop_duplicates(ogv_routes)

In [83]:
LINK_INPUT = "anyoldinput"
LINK_OUTPUT = "anyoldoutput"

In [88]:
new = []
dicto={}
for i in range(len(unique_routes)):
    dicto["PARAMETERS"] ={}
    route_expression = "' \\\"NO\\\"  = " + " or \\\"NO\\\"  = ".join(list(map(str,unique_routes[i]))) + "\\n'"
    dicto["PARAMETERS"]["INPUT"] = LINK_INPUT
    dicto["PARAMETERS"]["EXPRESSION"] = route_expression
    dicto["OUTPUTS"] = {}
    dicto["OUTPUTS"]["OUTPUT"] = LINK_OUTPUT
    dicto["OUTPUTS"]["FAIL_OUTPUT"] = LINK_OUTPUT
    new.append(dicto)

In [93]:
filename = "test.json"
with open(filename, "w") as f:
    json.dump(new, f)

export_results(unique_routes, "unique_routes")
export_results(ogv_unique_routes, "ogv_unique_routes")